In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error

In [2]:
# -Training Dataset-
dataset = np.load('datasets/dataset_train_3.npz')
dataset_x = dataset['X']
dataset_y = dataset['Y']

# -Validation Dataset-
test_dataset = np.load("datasets/dataset_test.npz") 
test_dataset_x = test_dataset['X']
test_dataset_y = test_dataset['Y']
test_dataset_y_resh = np.reshape(test_dataset_y, (-1, 1)) 

In [3]:
class CNN:

    def __init__(self, name="CNN", writer_name="CNN",
                 l1_num_filt = 16, l1_window = [4,4], l1_strides = [1,1],
                 l2_num_filt = 32, l2_window = [2,2], l2_strides = [1,1],
                 padding_type = "VALID",
                 fc_num_units = 256):
        
        with tf.variable_scope(name):
            # Batch of inputs (game states, one-hot encoded)
            self.X = tf.placeholder(tf.float32, [None, 13, 26, 9], name="X") # type tf.float32 is needed for the rest of operations

            # Batch of outputs (correct predictions of number of actions)
            self.Y_corr = tf.placeholder(tf.float32, [None, 1], name="Y")


            """
            First convnet:
            """
            
            # Padding = "VALID" -> no padding, "SAME" -> padding to keep the output dimension the same as the input one
            
            self.conv1 = tf.layers.conv2d(inputs = self.X,
                                         filters = l1_num_filt,
                                         kernel_size = l1_window,
                                         strides = l1_strides,
                                         padding = padding_type,
                                         activation = tf.nn.relu,
                                         use_bias = True,
                                         kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                         name = "conv1")
   
            """
            Second convnet:
            """
    
            self.conv2 = tf.layers.conv2d(inputs = self.conv1,
                                         filters = l2_num_filt,
                                         kernel_size = l2_window,
                                         strides = l2_strides,
                                         padding = padding_type,
                                         activation = tf.nn.relu,
                                         use_bias = True,
                                         kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                         name = "conv2")        
            
            # Flatten output of conv layers
            
            self.flatten = tf.contrib.layers.flatten(self.conv2)
            
            # Fully connected layer
            
            self.fc = tf.layers.dense(inputs = self.flatten,
                                  units = fc_num_units,
                                  activation = tf.nn.relu,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                  name="fc")
            
            # Output Layer
            
            self.output = tf.layers.dense(inputs = self.fc, 
                                          kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                          units = 1, 
                                          activation=None)
            
            # Train
            
            self.loss = tf.reduce_mean(tf.square(self.output - self.Y_corr), name="loss") # Quadratic loss
            
            self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001, name="optimizer")
            self.train_op = self.optimizer.minimize(self.loss, name="train_op")
            
            # Summaries
            self.train_loss_sum = tf.summary.scalar('train_loss', self.loss) # Training loss
            self.test_loss_sum = tf.summary.scalar('test_loss', self.loss) # Validation loss
            
            self.writer = tf.summary.FileWriter("ModeloCNN_log/" + writer_name)
            self.writer.add_graph(tf.get_default_graph())


In [4]:
CNN_test = CNN(name="CNN", writer_name="CNN_l1_(n=4,w=4,s=4)_l2_(n=8,w=2,s=2)_pad=V_fc_units=32",
               l1_num_filt = 4, l1_window = [4,4], l1_strides = [4,4],
               l2_num_filt = 8, l2_window = [2,2], l2_strides = [2,2],
               padding_type = "VALID",
               fc_num_units = 32)

In [5]:
# Number of epochs
num_epochs = 150
# Minibatch size
batch_size = 128

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # Initialize all variables
    
    #output = sess.run(CNN_test.output, feed_dict={CNN_test.X:test_dataset_x, CNN_test.Y_corr:test_dataset_y_resh})
    #print(output)
    
    rkf = RepeatedKFold(n_splits=int(len(dataset_y) // batch_size),
                      n_repeats=num_epochs) # Get randomized indexes for minibatches

    # Feed Dict for validation (uses validation set)
    data_dict_test = {CNN_test.X:test_dataset_x, CNN_test.Y_corr:test_dataset_y_resh}
        
    # Train the model
        
    it = 0
        
    for _, batch_index in rkf.split(dataset_y):
        batch_x = np.take(dataset_x, batch_index, axis=0) # Obtain current training batch
        batch_y = np.take(dataset_y, batch_index)
        batch_y = np.reshape(batch_y, (-1, 1))
            
        data_dict = {CNN_test.X:batch_x, CNN_test.Y_corr:batch_y}
           
        sess.run(CNN_test.train_op, feed_dict=data_dict) # Execute one training step
            
        # Periodically check losses
        if it % 5 == 0:                
            sum1 = sess.run(CNN_test.test_loss_sum, feed_dict=data_dict_test)
            CNN_test.writer.add_summary(sum1, it)
            sum2 = sess.run(CNN_test.train_loss_sum, feed_dict=data_dict)
            CNN_test.writer.add_summary(sum2, it)   

        it += 1
    
    # --- After training ---
        
    # Compute MAE
            
    dataset_y_reshaped = np.reshape(dataset_y, (-1, 1))   
    data_dict = {CNN_test.X:dataset_x}
        
    y_pred = sess.run(CNN_test.output, feed_dict=data_dict)
        
    mae_base = mean_absolute_error(dataset_y, np.repeat(np.mean(dataset_y), dataset_y.shape[0]))
    mae_model = mean_absolute_error(dataset_y, y_pred)
        
    print("\n\n---MAE TRAIN---\n\n")
    print("Model:", mae_model)
    print("Baseline:", mae_base)
        
    # ----- VALIDATION -----

    # Compute predictions
    data_dict = {CNN_test.X:test_dataset_x}
    test_pred = sess.run(CNN_test.output, feed_dict=data_dict)
        
    mae_base_test = mean_absolute_error(test_dataset_y, np.repeat(np.mean(dataset_y), test_dataset_y.shape[0]))
    mae_model_test = mean_absolute_error(test_dataset_y, test_pred)
        
    print("\n\n---MAE VALIDATION--\n\n")
    print("Model:", mae_model_test)
    print("Baseline:", mae_base_test)
    



---MAE TRAIN---


Model: 5.376889455714331
Baseline: 6.7887650132224895


---MAE VALIDATION--


Model: 7.513050222279691
Baseline: 7.099567260502611
